# Deep fake detection challenge

#### Setup tensorflow to use GPU if available

In [ ]:
import tensorflow as tf
import os

gpu = False
# Use gpu if available
if gpu:
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    os.environ['CUDA_VISIBLE_DEVICES'] = '1'
    physical_devices = tf.config.list_physical_devices('GPU')
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    print(os.environ['CUDA_VISIBLE_DEVICES'])  # Check the value

#### Setup the seed for random number generators to ensure reproducibility

In [ ]:
import random
import numpy as np
# Set seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Pre-Procesing

### Step 1

- Load the data from the directory
- Extract 20 random frames from each video
- Use MTcnn to detect faces in the frames. If no face is detected, discard the frame. If more than one face is detected, take the first face since both faces have been faked similarly.
- Resize the frames to 224x224
- Save the frames in a new directory and create a new image_label.json file

In [ ]:
import cv2
import imageio
import os
import random
import json
from mtcnn import MTCNN

# Paths
data_folder = "data"
output_folder = "data_images"

# If the output folder is not empty, skip processing
if os.path.exists(output_folder) and len(os.listdir(output_folder)) > 0:
    print("Skipping processing: 'data_images' folder is already populated.")
else:
    os.makedirs(output_folder, exist_ok=True)


    # Initialize MTCNN
    detector = MTCNN()

    # Parameters
    num_frames = 20
    frame_size = (224, 224)

    # Load labels from the existing metadata
    with open("data/metadata.json", "r") as f:
        video_labels = json.load(f)

    # Dictionary to store image-label mappings
    image_labels = {}

    # Process videos
    for video_name in os.listdir(data_folder):
        video_path = os.path.join(data_folder, video_name)

        if video_name.endswith(".mp4"):
            reader = imageio.get_reader(video_path, "ffmpeg")
            total_frames = reader.count_frames()

            # Select frame indices
            selected_frame_indices = sorted(random.sample(range(total_frames), num_frames))

            # Extract frames and process
            for i, frame in enumerate(reader):
                if i in selected_frame_indices:
                    # Detect faces in the frame and choose the first face
                    faces = detector.detect_faces(frame)
                    
                    # If no faces are found, skip this frame
                    if len(faces) == 0:
                        continue

                    # Get the bounding box of the first face
                    face = faces[0]
                    x, y, w, h = face['box']

                    # Crop the face region
                    face_crop = frame[y:y+h, x:x+w]
                    
                    if face_crop.size == 0:  # Ensure the cropped face is valid
                        continue
                    
                    # Resize the cropped face to 224x224
                    face_resized = cv2.resize(face_crop, frame_size)

                    # Generate a unique image name for the face
                    image_name = f"{video_name.split('.')[0]}_{i}.jpg"
                    image_path = os.path.join(output_folder, image_name)

                    # Save the resized face image
                    cv2.imwrite(image_path, face_resized)

                    # Store the label mapping
                    video_metadata = video_labels.get(video_name)
                    if video_metadata:
                        image_labels[image_name] = video_metadata["label"]

            reader.close()

    # Save the label mappings to a JSON file
    with open("image_labels.json", "w") as f:
        json.dump(image_labels, f, indent=4)

    print("Processing complete. Images saved in 'data_images', labels in 'image_labels.json'.")



### Image Label Counts Plot
- The plot above visualizes the count of `REAL` and `FAKE` images extracted from the videos.


In [ ]:
import matplotlib.pyplot as plt

image_labels = json.load(open("image_labels.json", "r"))
# Count the number of REAL and FAKE images outside of the loop
real_images_count = sum(1 for label in image_labels.values() if label == "REAL")
fake_images_count = sum(1 for label in image_labels.values() if label == "FAKE")

# Plotting the results
labels = ['REAL', 'FAKE']
counts = [real_images_count, fake_images_count]

plt.bar(labels, counts, color=['green', 'red'])
plt.xlabel('Image Type')
plt.ylabel('Count')
plt.title('REAL vs FAKE Image Counts')
plt.show()

### Step 2

In [ ]:
# load the images and labels and set it to a tf.data.Dataset

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load the image-label mappings
image_labels = json.load(open("image_labels.json", "r"))

fakes = np.empty((0, 2), dtype=object)
reals = np.empty((0, 2), dtype=object)

for image, label in image_labels.items():
    if label == "FAKE":
        fakes = np.append(fakes, np.array([[image, label]]), axis=0)
    else:
        reals = np.append(reals, np.array([[image, label]]), axis=0)
        
data = np.vstack((fakes, reals))

#### Perorm train-test split while stratifying on the labels


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42, stratify=data[:, 1])

train, val = train_test_split(train, test_size=0.2, random_state=42, stratify=train[:, 1])


#### Plot the total number of images in each class


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)

unique_labels, label_counts = np.unique(train[:, 1], return_counts=True)
axes[0].bar(unique_labels, label_counts, edgecolor='black')
axes[0].set_xticks(unique_labels)
axes[0].set_xticklabels(['Fakes', 'Real'])
axes[0].set_xlabel('Label')
axes[0].set_ylabel('Number of Images')
axes[0].set_title('Number of Images per Label in Training Set')

unique_labels, label_counts = np.unique(val[:, 1], return_counts=True)
axes[1].bar(unique_labels, label_counts, edgecolor='black')
axes[1].set_xticks(unique_labels)
axes[0].set_xticklabels(['Fakes', 'Real'])
axes[1].set_xlabel('Label')
axes[1].set_ylabel('Number of Images')
axes[1].set_title('Number of Images per Label in Validation Set')

unique_labels, label_counts = np.unique(test[:, 1], return_counts=True)
axes[2].bar(unique_labels, label_counts, edgecolor='black')
axes[2].set_xticks(unique_labels)
axes[0].set_xticklabels(['Fakes', 'Real'])
axes[2].set_xlabel('Label')
axes[2].set_ylabel('Number of Images')
axes[2].set_title('Number of Images per Label in Test Set')

# Display the plot
plt.show()

#### Save the images under train, test and val folders with the respective labels in the folder name

In [ ]:
from PIL import Image

def save_images(data, base_folder):
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    for img_path, label in data:
        label_folder = os.path.join(base_folder, str(label))
        if not os.path.exists(label_folder):
            os.makedirs(label_folder)
        img = Image.open(img_path)
        img.save(os.path.join(label_folder, os.path.basename(img_path)))

if os.path.exists('train') and os.path.exists('val') and os.path.exists('test') and len(os.listdir(output_folder)) > 0:
    print("Skipping processing: 'train, val, test' folder is already populated.")
else:
    # Save the images for each dataset
    save_images(train, 'train')
    save_images(val, 'val')
    save_images(test, 'test')

#### Load the images from the folders and perform data augmentation


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values to [0, 1]
    rotation_range=20,           # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,       # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,      # Randomly shift images vertically by 20% of the height
    shear_range=0.2,             # Apply shearing transformations
    zoom_range=0.2,              # Randomly zoom in or out by 20%
    horizontal_flip=True,        # Randomly flip images horizontally
    fill_mode='nearest'          # Fill missing pixels after transformations
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
# Create generators
train_generator = train_datagen.flow_from_directory(
    'train',                   
    target_size=(224, 224),     
    batch_size=32,              
    class_mode='categorical'    
)

val_generator = val_datagen.flow_from_directory(
    'val', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'test', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)